# Module 7 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# Unification

This is actually Part I of a two part assignment. In a later module, you'll implement a Forward Planner. In order to do that, however, you need to have a unifier. It is important to note that you *only* need to implement a unifier. Although the module talked about resolution, you do not need to implement anything like "standardizing apart". From the unifier's point of view, that should already have been done.

Unification is simply the *syntactic* balancing of expressions. There are only 3 kinds of expressions: constants, lists and (logic) variables. Constants and lists are only equal to each other if they're exactly the same thing or can be made to be the same thing by *binding* a value to a variable.

It really is that simple...expressions must be literally the same (identical) except if one or the other (or both) has a variable in that "spot".

## S-Expressions

With that out of the way, we need a language with which to express our constants, variables and predicates and that language will be based on s-expressions.

**constants** - There are two types of constants, values and predicates. Values should start with an uppercase letter. Fred is a constant value, so is Barney and Food. Predicates are named using lowercase letters. loves is a predicate and so is hates. This is only a convention. Secret: your code does not need to treat these two types of constants differently.

**variables** - these are named using lowercase letters but always start with a question mark. ?x is a variable and so is ?yum. This is not a convention.

**expressions (lists)** - these use the S-expression syntax a la LISP. (loves Fred Wilma) is an expression as is (friend-of Barney Fred) and (loves ?x ?y).

## Parsing

These functions are already included in the starter .py file.

In [1]:
import tokenize
from io import StringIO

This uses the above libraries to build a Lisp structure based on atoms. It is adapted from [simple iterator parser](http://effbot.org/zone/simple-iterator-parser.htm). The first function is the `atom` function.

In [2]:
def atom( next, token):
    if token[ 1] == '(':
        out = []
        token = next()
        while token[ 1] != ')':
            out.append( atom( next, token))
            token = next()
            if token[ 1] == ' ':
                token = next()
        return out
    elif token[ 1] == '?':
        token = next()
        return "?" + token[ 1]
    else:
        return token[ 1]

The next function is the actual `parse` function:

In [3]:
def parse(exp):
    src = StringIO(exp).readline
    tokens = tokenize.generate_tokens(src)
    return atom(tokens.__next__, tokens.__next__())

**Note** there was a change between 2.7 and 3.0 that "hid" the next() function in the tokenizer.

From a Python perspective, we want to turn something like "(loves Fred ?x)" to ["loves" "Fred" "?x"] and then work with the second representation as a list of strings. The strings then have the syntactic meaning we gave them previously.

In [4]:
parse("Fred")

'Fred'

In [5]:
parse( "?x")

'?x'

In [6]:
parse( "(loves Fred ?x)")

['loves', 'Fred', '?x']

In [7]:
parse( "(father_of Barney (son_of Barney))")

['father_of', 'Barney', ['son_of', 'Barney']]

## Unifier

Now that that's out of the way, here is the imperative pseudocode for unification. This is a classic recursive program with a number of base cases. Students for some reason don't like it, try the algorithm in the book, can't get it to work and then come back to this pseudocode.

Work through the algorithm by hand with your Self-Check examples if you need to but I'd suggest sticking with this implementation. It does work.

Here is imperative pseudocode for the algorithm:

```
def unification( exp1, exp2):
    # base cases
    if exp1 and exp2 are constants or the empty list:
        if exp1 = exp2 then return {}
        else return FAIL
    if exp1 is a variable:
        if exp1 occurs in exp2 then return FAIL
        else return {exp1/exp2}
    if exp2 is a variable:
        if exp2 occurs in exp1 then return FAIL
        else return {exp2/exp1}

    # inductive step
    first1 = first element of exp1
    first2 = first element of exp2
    result1 = unification( first1, first2)
    if result1 = FAIL then return FAIL
    apply result1 to rest of exp1 and exp2
    result2 = unification( rest of exp1, rest of exp2)
    if result2 = FAIL then return FAIL
    return composition of result1 and result2
```

`unification` can return...

1. `None` (if unification completely fails)
2. `{}` (the empty substitution list) or 
3. a substitution list that has variables as keys and substituted values as values, like {"?x": "Fred"}. 

Note that the middle case sometimes confuses people..."Sam" unifying with "Sam" is not a failure so you return {} because there were no variables so there were no substitutions. You do not need to further resolve variables. If a variable resolves to an expression that contains a variable, you don't need to do the substition.

If you think of a typical database table, there is a column, row and value. This Tuple is a *relation* and in some uses of unification, the "thing" in the first spot..."love" above is called the relation. If you have a table of users with user_id, username and the value then the relation is:

`(login ?user_id ?username)`

*most* of the time, the relation name is specified. But it's not impossible for the relation name to be represented by a variable:

`(?relation 12345 "smooth_operator")`

Your code should handle this case (the pseudocode does handle this case so all  you have to do is not futz with it).

Our type system is very simple. We can get by with just a few boolean functions. The first tests to see if an expression is a variable.

In [8]:
def is_variable( exp):
    return isinstance( exp, str) and exp[ 0] == "?"

In [9]:
is_variable( "Fred")

False

In [10]:
is_variable( "?fred")

True

The second tests to see if an expression is a constant:

In [11]:
def is_constant( exp):
    return isinstance( exp, str) and not is_variable( exp)

In [12]:
is_constant( "Fred")

True

In [13]:
is_constant( "?fred")

False

In [14]:
is_constant( ["loves", "Fred", "?wife"])

False

It might also be useful to know that:

<code>
type( "a")
&lt;type 'str'>
type( "a") == str
True
type( "a") == list
False
type( ["a"]) == list
True
</code>


You need to write the `unification` function described above. It should work with two expressions of the type returned by `parse`. See `unify` for how it will be called. It should return the result of unification for the two expressions as detailed above and in the book. It does not have to make all the necessary substitions (for example, if ?y is bound to ?x and 1 is bound to ?y, ?x doesn't have to be replaced everywhere with 1. It's enough to return {"?x":"?y", "?y":1}. You'll need to fix later.)

-----

**apply_result**<br>
The `apply_result` is a herlper function for `unification`. It is a recursive function that makes the required substitutions in the expression.

Parameters:
* **result** is the dictionary that contains keys that need to be substituted in the expression.
* **expr** is the list of strings we want to make substitutions to.

retuns:<br>
It does not return anything. It only updates the substitution in the `expr`

In [15]:
def apply_result(result, expr):
    for indx, value in enumerate(expr):
        if type(value) == list:
            apply_result(result, value)
        else:
            if value in result.keys():
                expr[indx] = result[value]

**occurs_check**<br>
The `occurs_check` is a herlper function for `unification`. It check if expr1 can be found in expr2.

Parameters:
* **expr1** is the expression we want to look for in expression2.
* **expr2** is the list we want to search expression 1 in.

retuns:<br>
It returns `True` if the expr1 is found in expr2 otherwise it returns `False`

In [16]:
def occurs_check(expr1, expr2):
    return True if expr1 in expr2 else False

**unification**<br>
The `unification` functions attempts to unify the two expression. Unification is simply the syntactic balancing of expressions. There are only 3 kinds of expressions: constants, lists and (logic) variables. Constants and lists are only equal to each other if they're exactly the same thing or can be made to be the same thing by binding a value to a variable.

Parameters:
* **list_expression1** is one of the two expressions we want to unify.
* **list_expression2** is one of the two expressions we want to unify.

retuns:<br>
It returns a `dictionary` of variables and substitutions if it is possible to unify the two expressions otherwise `fail` is returned.

In [17]:
def unification( list_expression1, list_expression2):
    if is_constant(list_expression1) == is_constant(list_expression2) == True or list_expression1 ==list_expression2 == []:  
        if list_expression1 == list_expression2: return {}
        else: return "fail"
    if is_variable(list_expression1):
        if occurs_check(list_expression1, list_expression2): return "fail"
        else: return {list_expression1: list_expression2}
    if is_variable(list_expression2):
        if occurs_check(list_expression2, list_expression1): return "fail"
        else: return {list_expression2: list_expression1}
    
    #inductive step
    first1 = list_expression1[0]
    first2 = list_expression2[0]
    result1 = unification(first1, first2)
    if result1 == "fail": return "fail"
    apply_result(result1, list_expression1), apply_result(result1, list_expression2)  #apply result 1 to both expressions
    result2 = unification(list_expression1[1:], list_expression2[1:])
    if result2 == "fail": return "fail"
    return {**result1, **result2}

**unify**<br>
The `unify` function makes the call to the `unification` function and parses both expressions.

Parameters:
* **list_expression1** is one of the two expressions we want to apply unification on.
* **list_expression2** is one of the two expressions we want to apply unification on.

retuns:<br>
It returns a `dictionary` of variables and substitutions if it is possible to unify the two expressions otherwise `fail` is returned.

In [18]:
def unify( s_expression1, s_expression2):
    return unification( parse( s_expression1), parse( s_expression2))

## Test Cases

Use the expressions from the Self Check as your test cases plus generate **five (5)** additional test cases of your own making.

**Note** If you see the error,

```
tokenize.TokenError: ('EOF in multi-line statement', (2, 0))
```
You most likely have unbalanced parentheses in your s-expression.

In [19]:
test_cases = [["Fred" ,"Barney", "fail"],    
              ["Pebbles" ,"Pebbles", {}],
              ["(quarry_worker Fred)" ,"(quarry_worker ?x)", {'?x': 'Fred'}],
              ["(son Barney ?x)" ,"(son ?y Bam_Bam)", {'?y': 'Barney', '?x': 'Bam_Bam'}],
              ["(married ?x ?y)" ,"(married Barney Wilma)",  {'?x': 'Barney', '?y': 'Wilma'}],
              ["(son Barney ?x)" ,"(son ?y (son Barney))", {'?y': 'Barney', '?x': ['son', 'Barney']}],
              ["(son Barney ?x)" ,"(son ?y (son ?y))", {'?y': 'Barney', '?x': ['son', 'Barney']}],
              ["(son Barney Bam_Bam)" ,"(son ?y (son Barney))", "fail"],
              ["(loves Fred Fred) " ,"(loves ?x ?x)", {'?x': 'Fred'}],
              ["(future George Fred)" ,"(future ?y ?y)", "fail"],  
            # My test casses
              ["(Fred (son Pebbles))" ,"(?y (son ?z))", {'?y': 'Fred', '?z': 'Pebbles'}],
              ["(Fred (Wilma (loves)))" ,"(Fred (?y loves))", "fail"],
              ["(Fred Barney (George ?x))" ,"(?x ?y ?z)", {'?x': 'Fred', '?y': 'Barney', '?z': ['George', 'Fred']}],
              ["(?x Barney)" ,"(?x Barney)", "fail"],
              ["(?y Fred ?y (Bam_Bam))", "(Barney ?x ?y (?z))", {'?y': 'Barney', '?x': 'Fred', '?z': 'Bam_Bam'}]]

col1, col2, col3 = "Expr1", "Expr2", "Result"
print(f"-"*90 +"\n"+f"{col1:{25}} | {col2:{25}} | {col3:{25}}","\n" + "-"*90)
for case in test_cases:
    exp1, exp2, expected = case
    actual = unify(exp1, exp2)
    print(f"{exp1:<25} | {exp2:25} | {actual}")
    assert(actual == expected)

------------------------------------------------------------------------------------------
Expr1                     | Expr2                     | Result                    
------------------------------------------------------------------------------------------
Fred                      | Barney                    | fail
Pebbles                   | Pebbles                   | {}
(quarry_worker Fred)      | (quarry_worker ?x)        | {'?x': 'Fred'}
(son Barney ?x)           | (son ?y Bam_Bam)          | {'?y': 'Barney', '?x': 'Bam_Bam'}
(married ?x ?y)           | (married Barney Wilma)    | {'?x': 'Barney', '?y': 'Wilma'}
(son Barney ?x)           | (son ?y (son Barney))     | {'?y': 'Barney', '?x': ['son', 'Barney']}
(son Barney ?x)           | (son ?y (son ?y))         | {'?y': 'Barney', '?x': ['son', 'Barney']}
(son Barney Bam_Bam)      | (son ?y (son Barney))     | fail
(loves Fred Fred)         | (loves ?x ?x)             | {'?x': 'Fred'}
(future George Fred)      | (future ?y

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.